In [ ]:
# Dan Lee
# CMSC 491 Computer Vision
# Assignment #2
# Due Date: 4/14/19
# Late Submission on 4/15/19 - which leaves me 3 more passes for the semester

import numpy as np # basic array manipulation
import scipy as sc # submodule dedicated to image processing (n-dimensional images) 
from PIL import Image # for reading images and converting images from numpy arrays
from matplotlib import pyplot as plt # for rendering the images 
import math # for basic math functions that do not manipulate arrays
import imageio # for a more convenient way writing images to files (PLEASE REFER TO PDF )
import cv2 # like TA said, ONLY used for the smaller functions

%matplotlib inline

# REMINDER TO SELF BEFORE SUBMITTING: 'Kernel > Restart' (restart the kernel) and then 'Cell > Run All' (run the script)

In [ ]:
# THIS CELL IS A PART OF FILE THAT THE TA GAVE TO US
# may or may not use, because of bugs

import numpy as np
from skimage.color import rgb2gray
from scipy.signal import convolve2d
from scipy.ndimage import rank_filter
from scipy.ndimage import filters
from scipy.stats import norm

def gen_dgauss(sigma):
    """
    Generates the horizontally and vertically differentiated Gaussian filter

    Parameters
    ----------
    sigma: float
        Standard deviation of the Gaussian distribution

    Returns
    -------
    Gx: numpy.ndarray
        First degree derivative of the Gaussian filter across rows
    Gy: numpy.ndarray
        First degree derivative of the Gaussian filter across columns
    """
    f_wid = 4 * np.floor(sigma)
    G = norm.pdf(np.arange(-f_wid, f_wid + 1),
                 loc=0, scale=sigma).reshape(-1, 1)
    G = G.T * G
    Gx, Gy = np.gradient(G)

    Gx = Gx * 2 / np.abs(Gx).sum()
    Gy = Gy * 2 / np.abs(Gy).sum()

    return Gx, Gy


def find_sift(I, circles, enlarge_factor=1.5):
    """
    Compute non-rotation-invariant SITF descriptors of a set of circles

    Parameters
    ----------
    I: numpy.ndarray
        Image
    circles: numpy.ndarray
        An array of shape `(ncircles, 3)` where ncircles is the number of
        circles, and each circle is defined by (x, y, r), where r is the radius
        of the cirlce
    enlarge_factor: float
        Factor which indicates by how much to enlarge the radius of the circle
        before computing the descriptor (a factor of 1.5 or large is usually
        necessary for best performance)

    Returns
    -------
    sift_arr: numpy.ndarray
        Array of SIFT descriptors of shape `(ncircles, 128)`
    """
    assert circles.ndim == 2 and circles.shape[1] == 3, \
        'Use circles array (keypoints array) of correct shape'
    I = I.astype(np.float64)
    if I.ndim == 3:
        I = rgb2gray(I)

    NUM_ANGLES = 8
    NUM_BINS = 4
    NUM_SAMPLES = NUM_BINS * NUM_BINS
    ALPHA = 9
    SIGMA_EDGE = 1

    ANGLE_STEP = 2 * np.pi / NUM_ANGLES
    angles = np.arange(0, 2 * np.pi, ANGLE_STEP)

    height, width = I.shape[:2]
    num_pts = circles.shape[0]

    sift_arr = np.zeros((num_pts, NUM_SAMPLES * NUM_ANGLES))

    Gx, Gy = gen_dgauss(SIGMA_EDGE)

    Ix = convolve2d(I, Gx, 'same')
    Iy = convolve2d(I, Gy, 'same')
    I_mag = np.sqrt(Ix ** 2 + Iy ** 2)
    I_theta = np.arctan2(Ix, Iy + 1e-12)

    interval = np.arange(-1 + 1/NUM_BINS, 1 + 1/NUM_BINS, 2/NUM_BINS)
    gridx, gridy = np.meshgrid(interval, interval)
    gridx = gridx.reshape((1, -1))
    gridy = gridy.reshape((1, -1))

    I_orientation = np.zeros((height, width, NUM_ANGLES))

    for i in range(NUM_ANGLES):
        tmp = np.cos(I_theta - angles[i]) ** ALPHA
        tmp = tmp * (tmp > 0)

        I_orientation[:, :, i] = tmp * I_mag

    for i in range(num_pts):
        cx, cy = circles[i, :2]
        r = circles[i, 2]

        gridx_t = gridx * r + cx
        gridy_t = gridy * r + cy
        grid_res = 2.0 / NUM_BINS * r

        x_lo = np.floor(np.max([cx - r - grid_res / 2, 0])).astype(np.int32)
        x_hi = np.ceil(np.min([cx + r + grid_res / 2, width])).astype(np.int32)
        y_lo = np.floor(np.max([cy - r - grid_res / 2, 0])).astype(np.int32)
        y_hi = np.ceil(
            np.min([cy + r + grid_res / 2, height])).astype(np.int32)

        grid_px, grid_py = np.meshgrid(
            np.arange(x_lo, x_hi, 1),
            np.arange(y_lo, y_hi, 1))
        grid_px = grid_px.reshape((-1, 1))
        grid_py = grid_py.reshape((-1, 1))

        dist_px = np.abs(grid_px - gridx_t)
        dist_py = np.abs(grid_py - gridy_t)

        weight_x = dist_px / (grid_res + 1e-12)
        weight_x = (1 - weight_x) * (weight_x <= 1)
        weight_y = dist_py / (grid_res + 1e-12)
        weight_y = (1 - weight_y) * (weight_y <= 1)
        weights = weight_x * weight_y

        curr_sift = np.zeros((NUM_ANGLES, NUM_SAMPLES))
        for j in range(NUM_ANGLES):
            tmp = I_orientation[y_lo:y_hi, x_lo:x_hi, j].reshape((-1, 1))
            curr_sift[j, :] = (tmp * weights).sum(axis=0)
        sift_arr[i, :] = curr_sift.flatten()

    tmp = np.sqrt(np.sum(sift_arr ** 2, axis=-1))
    if np.sum(tmp > 1) > 0:
        sift_arr_norm = sift_arr[tmp > 1, :]
        sift_arr_norm /= tmp[tmp > 1].reshape(-1, 1)

        sift_arr_norm = np.clip(sift_arr_norm, sift_arr_norm.min(), 0.2)

        sift_arr_norm /= np.sqrt(
            np.sum(sift_arr_norm ** 2, axis=-1, keepdims=True))

        sift_arr[tmp > 1, :] = sift_arr_norm

    return sift_arr



if __name__ == '__main__':
    Gx, Gy = gen_dgauss(3.2)
    print(f'Gx.shape: {Gx.shape}')
    I = np.random.random((480, 640, 3)) * 255
    circles = np.vstack([
        np.random.randint(1, 480, 25),
        np.random.randint(1, 640, 25),
        15 * np.random.random(25)]).T

    sift_arr = find_sift(I, circles)
    print(sift_arr.shape)

#     cim, r, c = harris(I, 3.2, thresh=5, radius=3)

#     print(f'cim.shape: {cim.shape}')

In [ ]:
# 1. Load both images - convert to grayscale and double
imgLeft = np.float32(np.array(Image.open('uttower_left.JPG').convert("L")))
imgRight = np.float32(np.array(Image.open('uttower_right.JPG').convert("L"))) 


In [ ]:
# 2. Detect feature points in both images with harris corner detection function.
# cimL, rL, cL = harris(grayL, 2, 0.05, 2)
# create coordinates from rL and cL - a total of 4898 coordinates exist
# filtered_coordsL = np.stack((rL, cL), axis=-1)
# convert from ndarray to list for computation on coordinates
# filtered_coordsL.tolist()
# print(len(filtered_coordsL))

# Due to issues with matching b/w coords extracted from the harris function in util
# provided by the TA I am using another harris detection function - using another harris implementation found on the web

def harris(im,min_dist=10,threshold=0.1):
    # derivatives
    imx = np.zeros(im.shape)
    filters.gaussian_filter(im, (3,3), (0,1), imx)
    imy = np.zeros(im.shape)
    filters.gaussian_filter(im, (3,3), (1,0), imy)
    # compute components of the Harris matrix
    Wxx = filters.gaussian_filter(imx*imx,3)
    Wxy = filters.gaussian_filter(imx*imy,3)
    Wyy = filters.gaussian_filter(imy*imy,3)
    # determinant and trace
    Wdet = Wxx*Wyy - Wxy**2
    Wtr = Wxx + Wyy
    harrisim = (Wdet / Wtr)
    # find top corner candidates above a threshold
    corner_threshold = harrisim.max() * threshold
    harrisim_t = (harrisim > corner_threshold) * 1
    # get coordinates of candidates
    coords = np.array(harrisim_t.nonzero()).T
    # ...and their values
    candidate_values = [harrisim[c[0],c[1]] for c in coords]
    # sort candidates
    index = np.argsort(candidate_values)
    # store allowed point locations in array
    allowed_locations = np.zeros(harrisim.shape)
    allowed_locations[min_dist:-min_dist,min_dist:-min_dist] = 1
    # select the best points taking min_distance into account
    filtered_coords = []
    for i in index:
        if allowed_locations[coords[i,0],coords[i,1]] == 1:
            filtered_coords.append(coords[i])
            allowed_locations[(coords[i,0]-min_dist):(coords[i,0]+min_dist),
            (coords[i,1]-min_dist):(coords[i,1]+min_dist)] = 0
    return filtered_coords

In [ ]:
filtered_coordsL = harris(imgLeft,6)
print(len(filtered_coordsL))
plt.figure()
plt.gray()
plt.imshow(imgLeft)
plt.plot([p[1] for p in filtered_coordsL],[p[0] for p in filtered_coordsL],"*")
plt.title('harris corners plot for LEFT IMG')
plt.show()

In [ ]:
filtered_coordsR = harris(imgRight,6)
print(len(filtered_coordsR))
plt.figure()
plt.gray()
plt.imshow(imgLeft)
plt.plot([p[1] for p in filtered_coordsR],[p[0] for p in filtered_coordsR],"*")
plt.title('harris corners plot for RIGHT IMG')
plt.show()

In [ ]:
def formdescriptors(image,filtered_coords,wid=5):
    desc = []
    for coords in filtered_coords:
        patch = image[coords[0]-wid:coords[0]+wid+1,
        coords[1]-wid:coords[1]+wid+1].flatten()
        desc.append(patch)
    return desc

def dist2(x, c):
    """
    Calculates squared distance between two sets of points.

    Parameters
    ----------
    x: numpy.ndarray
        Data of shape `(ndata, dimx)`
    c: numpy.ndarray
        Centers of shape `(ncenters, dimc)`

    Returns
    -------
    n2: numpy.ndarray
        Squared distances between each pair of data from x and c, of shape
        `(ndata, ncenters)`
    """
    assert x.shape[1] == c.shape[1], \
        'Data dimension does not match dimension of centers'

    x = np.expand_dims(x, axis=0)  # new shape will be `(1, ndata, dimx)`
    c = np.expand_dims(c, axis=1)  # new shape will be `(ncenters, 1, dimc)`

    # We will now use broadcasting to easily calculate pairwise distances
    n2 = np.sum((x - c) ** 2, axis=-1)

    return n2

# according to Question 5: You can select all pairs whose descriptor distances 
# are below a specified threshold, or select the top few hundred descriptor pairs 
# with the smallest pairwise distances.

def putMatch(desc1,desc2,threshold=0.5):
    n = len(desc1[0])
    
    # Question 4: Compute distances between every descriptor in one image and every descriptor in the other image.
    # will be using the provided dist2 function
    d = -np.ones((len(desc1),len(desc2)))
    
    for i in range(len(desc1)):
        for j in range(len(desc2)):
            
            d1 = (desc1[i] - np.mean(desc1[i])) / np.std(desc1[i])
            
            d2 = (desc2[j] - np.mean(desc2[j])) / np.std(desc2[j])
            
            # "Alternatively, experiment with computing normalized correlation"
            #src: https://cseweb.ucsd.edu/classes/sp04/cse252b/notes/lec14/lec14.pdf
            ncc_value = np.sum(d1 * d2) / (n-1)
            if ncc_value > threshold:
                d[i,j] = ncc_value
                
    ndx = np.argsort(-d)
    matches = ndx[:,0]
    
    return matches

In [ ]:
wid = 5
desL = formdescriptors(imgLeft,filtered_coordsL,wid)
desR = formdescriptors(imgRight,filtered_coordsR,wid)
matches = putMatch(desL,desR) # LATER CONVERTED TO HOMOGRAPHY POINTS w/ Ransac
print(matches.shape)


In [ ]:
 # http://vision.cs.utexas.edu/378h-fall2015/slides/lecture14.pdf

# 6. Run RANSAC to estimate a homography mapping one image onto the other. Report the number of inliers and the average
# residual for the inliers (squared distance between the point coordinates in one image and the transformed coordinates of the
# matching point in the other image). Also, display the locations of inlier matches in both images.

# src: http://www.scipy.org/Cookbook/RANSAC -> ransac.py
# Homography fitting calls for homogeneous least squares. The solution to the homogeneous least squares system AX=0 is
# obtained from the SVD of A by the singular vector corresponding to the smallest singular value:
# [U,S,V]=svd(A); X = V(:,end);

class RansacModel(object):
    def __init__(self,debug=False):
        self.debug = debug
    def fit(self, data):
        """ Fit homography to four selected correspondences. """
        #########################################
        # transpose 
        data = data.T
        # from points
        fp = data[:3,:4]
        # target points
        tp = data[:3,:4]

        # fit homography and return
        m = np.mean(fp[:2], axis=1)
        maxstd = np.max(np.std(fp[:2], axis=1)) + 1e-9
        C1 = np.diag([1/maxstd, 1/maxstd, 1])
        C1[0][2] = -m[0]/maxstd
        C1[1][2] = -m[1]/maxstd
        fp = np.dot(C1,fp)

        m = np.mean(tp[:2], axis=1)
        maxstd = np.max(np.std(tp[:2], axis=1)) + 1e-9
        C2 = np.diag([1/maxstd, 1/maxstd, 1])
        C2[0][2] = -m[0]/maxstd
        C2[1][2] = -m[1]/maxstd
        tp = np.dot(C2,tp)

        # create matrix for linear method, 2 rows for each correspondence pair
        A = np.zeros((8,9))
        for i in range(4): # using the corners
            A[2*i] = [-fp[0][i],-fp[1][i],-1,0,0,0,tp[0][i]*fp[0][i],tp[0][i]*fp[1][i],tp[0][i]]
            A[2*i+1] = [0,0,0,-fp[0][i],-fp[1][i],-1,tp[1][i]*fp[0][i],tp[1][i]*fp[1][i],tp[1][i]]

        U,S,V = np.linalg.svd(A)
        H = V[8].reshape((3,3))
        # decondition
        H = np.dot(np.linalg.inv(C2),np.dot(H,C1))
        # normalize and return
        return H / H[2,2]
        
    def get_error( self, data, H):
        """ Apply homography to all correspondences,
        return error for each transformed point. """
        data = data.T
        fp = data[:3]
        tp = data[3:]
        # transform 
        fp_transformed = np.dot(H,fp)
        # normalize the new coordinates
        for i in range(3):
            fp_transformed[i] /= fp_transformed[2]
        
        # return error per point
        return np.sqrt( np.sum((tp-fp_transformed)**2,axis=0) )

def H_from_ransac(fp,tp,model,maxiter=1000,match_threshold=10):
    """ Robust estimation of homography H from point
    correspondences using RANSAC (ransac.py from
    http://www.scipy.org/Cookbook/RANSAC).
    input: fp,tp (3*n arrays) points in hom. coordinates. """
    
    import ransac #ransac.py
    
    # group corresponding points
    data = np.vstack((fp,tp))
    # compute H and return
    H,ransac_data = ransac.ransac(data.T,model,4,maxiter,match_threshold,10,return_all=True)
    
    return H, ransac_data["inliers"]

def make_homog(points):
    """ Convert a set of points (dim*n array) to
    homogeneous coordinates. """
    return np.vstack((points,np.ones((1,points.shape[1]))))

ransac = RansacModel()

fp = make_homog(imgLeft[:,:2].T)
fp = fp.astype(int)
tp = make_homog(imgRight[:,:2].T)
tp = tp.astype(int)

# https://tinyurl.com/yyn8wz2t
H,inliers = H_from_ransac(fp,tp,ransac)

print("# INLIERS",len(inliers))


In [ ]:
from scipy import ndimage
# 7. Warp one image onto the other using the estimated transformation. To do this, you will need to learn about maketform and
# imtransform functions.
# 1. Load both images - convert to grayscale and double
imgLeft = np.float32(np.array(Image.open('uttower_left.JPG').convert("L")))
imgRight = np.float32(np.array(Image.open('uttower_right.JPG').convert("L"))) 
H = H_from_ransac(fp,tp,ransac)[0]
# H = np.array([[1.4,0.05,-100],[0.05,1.5,-100],[0,0,1]]) #ransac H is not working ..

transformImgLeft = ndimage.affine_transform(imgLeft,H[:2,:2],(H[0,2],H[1,2]))
alpha = (transformImgLeft > 0)
imgWarp = (1-alpha)*imgRight + alpha*transformImgLeft

plt.figure()
plt.gray()
plt.imshow(imgWarpRansac)
plt.axis("equal")
plt.title("warped image w/ransac estimation")
plt.show()

In [ ]:
from scipy import ndimage
# 7. Warp one image onto the other using the estimated transformation. To do this, you will need to learn about maketform and
# imtransform functions.
# 1. Load both images - convert to grayscale and double
imgLeft = np.float32(np.array(Image.open('uttower_left.JPG').convert("L")))
imgRight = np.float32(np.array(Image.open('uttower_right.JPG').convert("L"))) 
# H = H_from_ransac(fp,tp,ransac)[0]
H = np.array([[1.4,0.05,-100],[0.05,1.5,-100],[0,0,1]]) #ransac H is not working ideally..

transformImgLeft = ndimage.affine_transform(imgLeft,H[:2,:2],(H[0,2],H[1,2]))
alpha = (transformImgLeft > 0)
imgWarp = (1-alpha)*imgRight + alpha*transformImgLeft
plt.figure()
plt.gray()
plt.imshow(imgWarp)
plt.axis("equal")
plt.title("warped image")
plt.show()

In [ ]:
# 8. Create a new image big enough to hold the panorama and composite the two images into it. 
# You can composite by simply averaging the pixel values where the two images overlap. 

def panorama(H,fromim,toim,padding=2400,delta=2400):
    # H from ransac will be used
    # check if images are grayscale or color
    is_color = len(fromim.shape) == 3
    
    # homography transformation for geometric_transform()
    def transf(p):
        p2 = np.dot(H,[p[0],p[1],1])
        return (p2[0]/p2[2],p2[1]/p2[2])
    
    if H[1,2]<0: # fromim is to the right
        # transform fromim
        if is_color:
            # pad the destination image with zeros to the right
            toim_t = np.hstack((toim,np.zeros((toim.shape[0],padding,3))))
            fromim_t = np.zeros((toim.shape[0],toim.shape[1]+padding,toim.shape[2]))
            for col in range(3):
                fromim_t[:,:,col] = ndimage.geometric_transform(fromim[:,:,col],
                    transf,(toim.shape[0],toim.shape[1]+padding))
        else:
            # pad the destination image with zeros to the right
            toim_t = np.hstack((toim,np.zeros((toim.shape[0],padding))))
            fromim_t = ndimage.geometric_transform(fromim,transf,(toim.shape[0],toim.shape[1]+padding))
    else:
        # add translation to compensate for padding to the left
        H_delta = np.array([[1,0,0],[0,1,-delta],[0,0,1]])
        H = np.dot(H,H_delta)
        # transform fromim
        if is_color:
            # pad the destination image with zeros to the left
            toim_t = np.hstack((np.zeros((toim.shape[0],padding,3)),toim))
            fromim_t = np.zeros((toim.shape[0],toim.shape[1]+padding,toim.shape[2]))
            for col in range(3):
                fromim_t[:,:,col] = ndimage.geometric_transform(fromim[:,:,col],transf,(toim.shape[0],toim.shape[1]+padding))
        else:
            # pad the destination image with zeros to the left
            toim_t = np.hstack((np.zeros((toim.shape[0],padding)),toim))
            fromim_t = ndimage.geometric_transform(fromim,transf,(toim.shape[0],toim.shape[1]+padding))
            
    # blend and return (left image above right =-> then average the overlapping pixels)
    if is_color:
        # all non black pixels
        alpha = ((fromim_t[:,:,0] * fromim_t[:,:,1] * fromim_t[:,:,2] ) > 0)
        for col in range(3):
            toim_t[:,:,col] = fromim_t[:,:,col]*alpha + toim_t[:,:,col]*(1-alpha)
    else:
        alpha = (fromim_t < 0)
        toim_t = fromim_t*alpha + toim_t*(1-alpha)
        
    return toim_t



In [ ]:
# 1. Load both images - convert to grayscale and double
imgLeft = np.float32(np.array(Image.open('uttower_left.JPG').convert("L")))
imgRight = np.float32(np.array(Image.open('uttower_right.JPG').convert("L"))) 

H = H_from_ransac(fp,tp,ransac)[0]
# H = np.array([[1.4,0.05,-100],[0.05,1.5,-100],[0,0,1]]) #ransac H is not working ideally..

imgPanoGray = panorama(H,imgLeft,imgRight,10,500)

plt.figure()
plt.gray()
plt.imshow(imgPanoGray)
plt.title("pano in grayscale b/w left and right img")
plt.show()

In [ ]:
imgLeftC = np.float32(np.array(Image.open('uttower_left.JPG').convert("RGB")))
imgRightC = np.float32(np.array(Image.open('uttower_right.JPG').convert("RGB"))) 

# H = H_from_ransac(fp,tp,ransac)[0]
H = np.array([[1.4,0.05,-100],[0.05,1.5,-100],[0,0,1]]) #ransac H is not working ideally..
imgPanoColor = panorama(H,imgLeftC,imgRightC,10,10)

plt.figure()
plt.imshow(imgPanoColor)
plt.title("pano in color b/w left and right img")
plt.show()